In [ ]:
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
import time
from datetime import datetime, timedelta


In [ ]:
# Initialize Connection & Global Settings

In [ ]:
# ----------------------------------
#       INITIAL SETUP
# ----------------------------------

# Replace these credentials if needed (for certain brokers, you might not need login/password)
MT5_LOGIN = 12345678
MT5_PASSWORD = "your_password"
MT5_SERVER = "YourBrokerServer"  # e.g., "MetaQuotes-Demo"

# Symbols of interest
SYMBOLS = ["XAUUSD", "BTCUSD", "GBPUSD", "USTEC"]  # Adjust to match your broker's naming

# Risk settings
RISK_PER_TRADE = 0.01        # 1% per trade
MAX_OVERALL_RISK = 0.03      # 3% total open risk

# For example strategies, we’ll primarily check the 4-hour time frame
TIMEFRAME = mt5.TIMEFRAME_H4

# Connect to MetaTrader 5
if not mt5.initialize(login=MT5_LOGIN, password=MT5_PASSWORD, server=MT5_SERVER):
    print("Initialize() failed")
    mt5.shutdown()
else:
    print(f"Connected to MT5 Client (Version: {mt5.version()})")


## 3. Helper Functions
### 3.1 Data Retrieval

In [ ]:
def get_ohlc_data(symbol, timeframe, n=500):
    """
    Fetches the last `n` candles of OHLC data for the specified symbol/timeframe.
    Returns a pandas DataFrame with columns: [time, open, high, low, close, tick_volume].
    """
    rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, n)
    if rates is None:
        return None
    df = pd.DataFrame(rates)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df.set_index('time', inplace=True)
    return df


3.2 Indicator Calculations:

- Moving Average (EMA)
- ATR (for volatility-based stops)
- RSI or MACD (if you want additional signals)

In [ ]:
def ema(series, period=20):
    """Calculate Exponential Moving Average."""
    return series.ewm(span=period, adjust=False).mean()

def atr(df, period=14):
    """Calculate Average True Range."""
    data = df.copy()
    data['H-L'] = data['high'] - data['low']
    data['H-C'] = abs(data['high'] - data['close'].shift(1))
    data['L-C'] = abs(data['low'] - data['close'].shift(1))
    tr = data[['H-L','H-C','L-C']].max(axis=1)
    atr_ = tr.rolling(period).mean()
    return atr_

def rsi(series, period=14):
    """Calculate RSI."""
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))


## 4. Strategy Logic
Based on  plan:

1. Identify trend (e.g., using EMAs, price structure).
2. Confirm a retracement + rejection or a breakout.
3. Use ATR for stop-loss distance.
4. Size your position according to 1% risk.
5. Manage trades with trailing stops or partial take-profits.

4.1 Check Trend & Potential Signals

Use a 50 EMA and 200 EMA to determine if price is in an uptrend or downtrend. Then look for a simple “retrace and rejection” entry.

In [ ]:
def check_signals(df):
    """
    Given a DataFrame with OHLC data,
    return a dictionary containing potential 'BUY' or 'SELL' signals,
    along with recommended stop loss level, etc.
    """

    # Basic check to ensure we have enough data
    if df is None or len(df) < 200:
        return None

    df['EMA50'] = ema(df['close'], 50)
    df['EMA200'] = ema(df['close'], 200)
    df['ATR'] = atr(df, 14)
    df['RSI'] = rsi(df['close'], 14)

    # Identify the last candle’s data
    last_close = df['close'].iloc[-1]
    last_ema50 = df['EMA50'].iloc[-1]
    last_ema200 = df['EMA200'].iloc[-1]
    last_atr = df['ATR'].iloc[-1]
    last_rsi = df['RSI'].iloc[-1]

    # Determine trend based on EMA alignment
    # Uptrend if EMA50 > EMA200, Downtrend if EMA50 < EMA200
    trend = None
    if last_ema50 > last_ema200:
        trend = 'UP'
    elif last_ema50 < last_ema200:
        trend = 'DOWN'

    # Example: looking for a retracement if price is in an uptrend
    # We might use RSI or a candlestick pattern for final confirmation
    signal = None
    sl_price = None

    if trend == 'UP':
        # Check for RSI bounce above 50 or a bullish RSI divergence, etc.
        # For simplicity, let's say if RSI > 50 and last_close > EMA50, we consider a buy signal
        if last_rsi > 50 and last_close > last_ema50:
            signal = "BUY"
            # Stop loss placed below the recent swing low or 1 x ATR below the close
            # We'll do: sl_price = last_close - 1.0 * last_atr
            sl_price = last_close - 1.0 * last_atr
    elif trend == 'DOWN':
        # Check for RSI < 50 and last_close < EMA50 for a sell signal
        if last_rsi < 50 and last_close < last_ema50:
            signal = "SELL"
            # Stop loss placed above the recent swing high or 1 x ATR above the close
            sl_price = last_close + 1.0 * last_atr

    if signal:
        return {
            "signal": signal,
            "sl_price": sl_price,
            "atr": last_atr
        }
    else:
        return None


In [ ]:
### 4.2 Position Sizing & Order Placement

We’ll create a function to calculate lot size based on a 1% risk (or user-specified RISK_PER_TRADE)
